In [1]:
# Libraries
import numpy as np
import pandas as pd
import os
import glob
import re
import sys

import plotnine
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation, rc

In [2]:
# Setting directory
os.chdir("/Users/JYW/Desktop/Github/Data-Science-for-COVID-19")

# Load files
files = glob.glob('dataset/*.csv')

data = []
for i in files :
    i = i[8:]
    data.append(re.sub(".csv","",i))

for i in range(len(files)) :
    globals()[data[i]] = pd.read_csv(files[i])

data

C:\Users\JYW


['Case',
 'PatientInfo',
 'PatientRoute',
 'Region',
 'SearchTrend',
 'Time',
 'TimeAge',
 'TimeGender',
 'TimeProvince',
 'Weather']

## Data Merge

### 1. Case + Patient

In [3]:
merged_pat = pd.merge(PatientInfo,Case,
        how='left',
        on=('infection_case','province','city')
        )

# 일단 루트는 선우가 하고 있으니 몇번의 이동 변화가 있는지만 반영해서 넣어보자
PatientRouteSum = PatientRoute[['patient_id', 'date']].groupby('patient_id').count()

merged_pat = pd.merge(merged_pat, PatientRouteSum, on=["patient_id"], how="left")
merged_pat.rename(columns={'date':'MoveCount'}, inplace=True)
merged_pat.head()

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,...,confirmed_date,released_date,deceased_date,state,case_id,group,confirmed,latitude,longitude,MoveCount
0,1000000001,2.0,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,...,2020-01-23,2020-02-05,NaN,released,NaN,NaN,NaN,NaN,NaN,2.0
1,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,...,2020-01-30,2020-03-02,NaN,released,NaN,NaN,NaN,NaN,NaN,9.0
2,1000000003,6.0,male,1964.0,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,...,2020-01-30,2020-02-19,NaN,released,NaN,NaN,NaN,NaN,NaN,3.0
3,1000000004,7.0,male,1991.0,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,...,2020-01-30,2020-02-15,NaN,released,NaN,NaN,NaN,NaN,NaN,2.0
4,1000000005,9.0,female,1992.0,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,...,2020-01-31,2020-02-24,NaN,released,NaN,NaN,NaN,NaN,NaN,1.0


### 2-1. Case+Region

In [4]:
Case['code'] = Case['case_id'].astype('str').str[0:5].astype('int64')
Case['case_id_new'] = Case['case_id'].astype('str').str[5:8].astype('int64')

merged_CR = pd.merge(Region,Case[["code", "case_id_new", "group", "infection_case", "confirmed"]],
        how='left',
        on=('code')
        )

merged_CR.head()

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count,case_id_new,group,infection_case,confirmed
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739,1.0,True,Guro-gu Call Center,79.0
1,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739,2.0,True,Dongan Church,24.0
2,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739,3.0,True,Eunpyeong St. Mary's Hospital,14.0
3,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739,4.0,True,Seongdong-gu APT,13.0
4,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739,5.0,True,Jongno Community Center,10.0


### 2-2. Region + Weather => 추후 필요하면...

### 3. Time (Crawling 필요?)

In [5]:
Time.rename(columns={'confirmed':'total_confirmed', 'released':'total_released', 'deceased':'total_deceased', 'test':'total_test', 'negative':'total_negative'}, inplace=True)
merged_time = pd.merge(TimeProvince,Time,
        how='left',
        on=('date', 'time')
        )

merged_time.head()

,date,time,province,confirmed,released,deceased,total_test,total_negative,total_confirmed,total_released,total_deceased
0,2020-01-20,16,Seoul,0,0,0,1,0,1,0,0
1,2020-01-20,16,Busan,0,0,0,1,0,1,0,0
2,2020-01-20,16,Daegu,0,0,0,1,0,1,0,0
3,2020-01-20,16,Incheon,1,0,0,1,0,1,0,0
4,2020-01-20,16,Gwangju,0,0,0,1,0,1,0,0
